In [2]:
%load_ext autoreload
%autoreload 2
%load_ext autotime

import sys
sys.path.append("..")

from libs.dataset_readers.ssqa_span_reader import SSQASpanReader

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 4.28 ms


In [3]:
reader = SSQASpanReader("bert-base-chinese")

time: 3.7 s


In [4]:
dataset_path = "../data/ssqa_multiple_choice_span/test.json"
dataset = reader.read(dataset_path)

time: 1.07 s


In [5]:
len(dataset)

294

time: 7.32 ms


In [6]:
print(0, dataset[0])
print(1, dataset[1])
print(2, dataset[2])
print(3, dataset[3])
print(4, dataset[4])
# print(5, dataset[5])
# print(6, dataset[6])

0 Instance with fields:
 	 question_with_context: TextField of length 72 with text: 
 		[[CLS], 班, 级, 公, 约, 在, 何, 时, 决, 定, ？, [SEP], 班, 级, 自, 治, 就, 是, 透, 过, 班, 会, ，, 订, 定, 班, 级, 公, 约, 、,
		选, 举, 班, 级, 干, 部, 、, 讨, 论, 班, 级, 事, 务, 、, 安, 排, 班, 级, 活, 动, ，, 学, 习, 自, 己, 管, 理, 班, 级, 事, 务, ，, 让,
		班, 级, 表, 现, 得, 更, 好, 。, [SEP]]
 		and TokenIndexers : {'tokens': 'PretrainedTransformerIndexer'} 
 	 answer_span: SpanField with spans: (20, 21). 
 	 context_span: SpanField with spans: (12, 70). 
 	 metadata: MetadataField (print field.metadata to see specific information). 

1 Instance with fields:
 	 question_with_context: TextField of length 85 with text: 
 		[[CLS], 班, 级, 公, 约, 由, 谁, 来, 订, 定, ？, [SEP], 班, 级, 公, 约, 是, 全, 班, 同, 学, 共, 同, 订, 定, ，, 用, 以, 约, 束,
		每, 个, 人, 的, 行, 为, 。, 订, 定, 班, 级, 公, 约, 时, ，, 要, 考, 虑, 全, 班, 同, 学, 的, 需, 要, ，, 制, 定, 明, 确, 、, 公, 平,
		且, 具, 体, 可, 行, 的, 规, 则, ，, 让, 全, 班, 同, 学, 都, 能, 确, 实, 遵, 守, 。, [SEP]]
 		and TokenIndexers : {'tokens': 'PretrainedTransformerIndexer'} 
 	 an

In [ ]:
print(dataset[0], dataset[0]["metadata"].metadata)

---
## Test

In [23]:
!python -m pytest tests/dataset_readers/ssqa_singlespan_test.py

[autoreload of libs.dataset_readers.ssqa_singlespan failed: Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/opt/conda/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/opt/conda/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/opt/conda/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 728, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/workspace/GNN-based SSQA/libs/dataset_readers/ssqa_singlespan.py", line 26, in <module>
    class SSQASinglespanReader(DatasetReader):
  File "/home/user/.local/lib/python3.7/site-packages/alle

============================= test session starts ==============================
platform linux -- Python 3.7.7, pytest-6.0.1, py-1.9.0, pluggy-0.13.1
rootdir: /workspace/GNN-based SSQA
collected 1 item                                                               

tests/dataset_readers/ssqa_singlespan_test.py .                          [100%]

=============================== warnings summary ===============================
/opt/conda/lib/python3.7/site-packages/tensorflow/python/data/ops/iterator_ops.py:546
  /opt/conda/lib/python3.7/site-packages/tensorflow/python/data/ops/iterator_ops.py:546: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
    class IteratorBase(collections.Iterator, trackable.Trackable,

/opt/conda/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:106
  /opt/conda/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.

---
## Test create answer span position

In [24]:
from typing import Any, Dict, List, Tuple, Optional, Iterable

from allennlp.data.tokenizers import Token, PretrainedTransformerTokenizer
from allennlp_models.rc.dataset_readers.utils import char_span_to_token_span
from allennlp.common.util import sanitize_wordpiece

In [22]:
context = "We also support limiting the maximum length for the question. and rectangle"

tokenizer = PretrainedTransformerTokenizer("bert-base-cased", add_special_tokens=False)

def tokenize_slice(start: int, end: int) -> Iterable[Token]:
    text_to_tokenize = context[start:end]
    if start - 1 >= 0 and context[start - 1].isspace():
        prefix = "a "  # must end in a space, and be short so we can be sure it becomes only one token
        wordpieces = tokenizer.tokenize(prefix + text_to_tokenize)
        for wordpiece in wordpieces:
            if wordpiece.idx is not None:
                wordpiece.idx -= len(prefix)
        return wordpieces[1:]
    else:
        return tokenizer.tokenize(text_to_tokenize)

tokenized_context = []
token_start = 0
for i, c in enumerate(context):
    if c.isspace():
        for wordpiece in tokenize_slice(token_start, i):
            if wordpiece.idx is not None:
                wordpiece.idx += token_start
            tokenized_context.append(wordpiece)
        token_start = i + 1
for wordpiece in tokenize_slice(token_start, len(context)):
    if wordpiece.idx is not None:
        wordpiece.idx += token_start
    tokenized_context.append(wordpiece)
    
for tok in tokenized_context:
    print(tok, tok.idx)
    
# We 0
# also 3
# support 8
# limiting 16
# the 25
# maximum 29
# length 37
# for 44
# the 48
# question 52
# . 60
# and 62
# re 66
# ##ct 68
# ##ang 70
# ##le 73

We 0
also 3
support 8
limiting 16
the 25
maximum 29
length 37
for 44
the 48
question 52
. 60
and 62
re 66
##ct 68
##ang 70
##le 73


In [25]:
[
    (t.idx, t.idx + len(sanitize_wordpiece(t.text))) if t.idx is not None else None
    for t in tokenized_context
]

# [(0, 2),
#  (3, 7),
#  (8, 15),
#  (16, 24),
#  (25, 28),
#  (29, 36),
#  (37, 43),
#  (44, 47),
#  (48, 51),
#  (52, 60),
#  (60, 61),
#  (62, 65),
#  (66, 68),
#  (68, 70),
#  (70, 73),
#  (73, 75)]

[(0, 2),
 (3, 7),
 (8, 15),
 (16, 24),
 (25, 28),
 (29, 36),
 (37, 43),
 (44, 47),
 (48, 51),
 (52, 60),
 (60, 61),
 (62, 65),
 (66, 68),
 (68, 70),
 (70, 73),
 (73, 75)]

In [30]:
first_answer_offset = 16
answers = "limiting"

char_span_to_token_span(
    [
        (t.idx, t.idx + len(sanitize_wordpiece(t.text))) if t.idx is not None else None
        for t in tokenized_context
    ],
    (first_answer_offset, first_answer_offset + len(answers[0])),
)

# ((3, 3), True)

((3, 3), True)

In [34]:
len(tokenizer.sequence_pair_end_tokens)
# 1

1